In [1]:
#pip install ipywidgets matplotlib


In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
#import spacy

import json
import pickle


model = SentenceTransformer("all-MiniLM-L6-v2")

### Extraction of cases

In [3]:

with open("output_file_1.txt", 'r') as f:
    lines = f.readlines()
    

In [4]:


# Extract JSON data from the first line
first_line_json = json.loads(lines[0])
first_line_json


{'id': 1869772,
 'url': 'https://api.capapi.org/v1/cases/1869772/',
 'name': 'James Joseph STANDLEY, Jr. v. STATE of Arkansas',
 'name_abbreviation': 'Standley v. State',
 'decision_date': '1987-11-09',
 'docket_number': '',
 'first_page': '517',
 'last_page': '518',
 'citations': [{'cite': '293 Ark. 517', 'type': 'official'},
  {'cite': '738 S.W.2d 423', 'type': 'parallel'}],
 'volume': {'volume_number': '293',
  'barcode': '32044078577145',
  'url': 'https://api.capapi.org/v1/volumes/32044078577145/'},
 'reporter': {'id': 368,
  'full_name': 'Arkansas Reports',
  'url': 'https://api.capapi.org/v1/reporters/368/'},
 'court': {'name_abbreviation': 'Ark.',
  'name': 'Arkansas Supreme Court',
  'id': 8808,
  'slug': 'ark',
  'url': 'https://api.capapi.org/v1/courts/ark/'},
 'jurisdiction': {'name': 'Ark.',
  'id': 34,
  'slug': 'ark',
  'name_long': 'Arkansas',
  'url': 'https://api.capapi.org/v1/jurisdictions/ark/',
  'whitelisted': True},
 'cites_to': [{'cite': '738 S.W.2d 423'}],
 'fr

In [5]:

ls_cases=[]
f=open("output_file_1.txt", 'r')
    
# Extract opinions from subsequent lines
#for line in f.readlines():
    #print(line)
    #data = json.loads(line)
    #if(len(data['casebody']['data']['opinions']) > 0):
    #    ls_cases.append(data['casebody']['data']['opinions'][0]['text'])
    
for k in f.readlines():
    # display({mkeys[n]:kk for n,kk in enumerate(json.loads(k)['casebody']['data']['opinions'][0]['text'].split(" In Re:"))}['case'])
    if (len(json.loads(k)['casebody']['data']['opinions'])>0):
        # print(json.loads(k)['casebody']['data']['opinions'][0]['text'])
        ls_cases.append(json.loads(k)['casebody']['data']['opinions'][0]['text'])
    # print('-----')   


### Tokenization

In [6]:
#nlp=spacy.load("en_core_web_sm")

In [7]:
# ls_tokens=[]
# for f in ls_cases:
#     doc=nlp(f)
#     for token in doc:
#         # print(token.lemma_)
#         if (token.lemma_ not in ls_tokens):
#             ls_tokens.append(token.lemma_)

In [8]:
#ls_cases

### Encodage of cases

In [9]:
#dict_emb_cases={}
#for n,f in enumerate(ls_cases):
#    dict_emb_cases[n]=model.encode(f)

In [10]:

#dict_emb_cases

In [11]:


rs=open("backup_embs.pck","wb")
pickle.dump(dict_emb_cases,rs)
rs.close()


In [12]:
#dict_emb_cases

### Function to check the similarity

In [13]:

def checkSimCases(_new_Case='', _dict_emb_cases={}, _threshold=0.1):
    emb_new_case=model.encode(_new_Case)
    ls_cos_sim=[]
    for f in _dict_emb_cases.keys():
        ls_cos_sim.append(util.cos_sim(emb_new_case,_dict_emb_cases[f]).item())
    
    # print(ls_cos_sim)
    sim_cases=[]
    for n,f in enumerate(ls_cos_sim):
        if (f>=_threshold):
           sim_cases.append(n)
           
    return sim_cases
        

### Test for a new case

In [14]:

#new_case="In the case of State v. Johnson, the defendant, Mr. Mark Johnson, was charged with first-degree rape following an incident that occurred on May 15, 2023. The prosecution presented evidence indicating that Mr. Johnson forcibly assaulted and raped the victim, Ms. Emily Davis, in her apartment after breaking in during the late hours of the night. The defense argued that there was insufficient evidence to prove Mr. Johnson's guilt beyond a reasonable doubt, contending that the encounter was consensual. After a thorough trial, the jury found Mr. Johnson guilty of first-degree rape, and he was sentenced to twenty-five years in prison."


In [15]:

#ls_sim_cases = checkSimCases(_new_Case=new_case, _dict_emb_cases=dict_emb_cases, _threshold=0.75)
#selected_cases = [ls_cases[i] for i in ls_sim_cases]
#for case in selected_cases:
#    print(case)


In [16]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np


# Define widgets
new_case_text = widgets.Text(description='New Case:')
threshold_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Threshold:')
submit_button = widgets.Button(description='Submit')
output_text = widgets.Output()

# Define event handlers
def submit_button_clicked(b):
    with output_text:
        clear_output(wait=True)  # Clear previous output
        new_case = new_case_text.value
        threshold = threshold_slider.value
        ls_sim_cases = checkSimCases(new_case, dict_emb_cases, threshold)
        selected_cases = [ls_cases[i] for i in ls_sim_cases]
        if selected_cases:
            print("Similar Cases:")
            for case in selected_cases:
                print(case)
        else:
            print("No similar cases found.")


# Assign event handlers to widgets
submit_button.on_click(submit_button_clicked)

# Display widgets
display(new_case_text)
display(threshold_slider)
display(submit_button)
display(output_text)


Text(value='', description='New Case:')

FloatSlider(value=0.5, description='Threshold:', max=1.0, step=0.01)

Button(description='Submit', style=ButtonStyle())

Output()